In [7]:
import pandas
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt


In [38]:
df = pandas.read_csv('osu18_cerenkov_feat_mat.tsv', sep='\t')
df.head()

names = df['name']
del df['name']

df = df.astype(float)

labels = df['label']
del df['label']

labels = labels.fillna(0)
labels = labels.astype(int)


In [4]:
"""
trainDf = df[:10000]
testDf = df[10000:]


dtrain = xgb.DMatrix(trainDf, label=labels[:10000])
dtest = xgb.DMatrix(testDf, label=labels[10000:])


evallist = [(dtest, 'eval'), (dtrain, 'train')]


# param_dist = dict(max_depth=[7],
#                   learning_rate=[0.1],
#                   n_estimators=[40], 
#                   gamma=[10],
#                   scale_pos_weight=[1],
#                   base_score=[rare_event_rate],
#                   subsample=[1])

param = dict(max_depth=7,
            learning_rate=0.1,
            n_estimators=10,
            gamma=10,
            scale_pos_weight=1,
            base_score=0.5,
            subsample=1)

num_round = 20

bst = xgb.train(param, dtrain, num_round, evallist)
"""

In [87]:
split = 5000

trainX = df[:split]
trainY = labels[:split]
testX = df[split:]
testY = labels[split:]


In [88]:
param_dist = dict(max_depth=7,
            learning_rate=0.1,
            n_estimators=40,
            gamma=10,
            scale_pos_weight=1,
            base_score=rare_event_rate,
            subsample=1,
            objective= 'binary:logistic' )

#param_dist = { 'objective':'binary:logistic', 'n_estimators': 2 }

_RANDOM_STATE = 1337
# class_balance = len(y) / sum(y) - 1  # n_negative / n_positive
rare_event_rate = sum(trainY) / len(trainY)

clf = xgb.XGBClassifier(**param_dist, booster='gbtree', n_jobs=-1, random_state=_RANDOM_STATE)

clf.fit( trainX, trainY,
        eval_set=[(trainX, trainY), (testX, testY)],
        eval_metric='logloss',
        verbose=True)

evals_result = clf.evals_result()

[0]	validation_0-logloss:0.20067	validation_1-logloss:0.21709
[1]	validation_0-logloss:0.18645	validation_1-logloss:0.20856
[2]	validation_0-logloss:0.17646	validation_1-logloss:0.20216
[3]	validation_0-logloss:0.16889	validation_1-logloss:0.19707
[4]	validation_0-logloss:0.16230	validation_1-logloss:0.19397
[5]	validation_0-logloss:0.15735	validation_1-logloss:0.19066
[6]	validation_0-logloss:0.15270	validation_1-logloss:0.18846
[7]	validation_0-logloss:0.14872	validation_1-logloss:0.18748
[8]	validation_0-logloss:0.14578	validation_1-logloss:0.18584
[9]	validation_0-logloss:0.14423	validation_1-logloss:0.18485
[10]	validation_0-logloss:0.14154	validation_1-logloss:0.18336
[11]	validation_0-logloss:0.13905	validation_1-logloss:0.18218
[12]	validation_0-logloss:0.13698	validation_1-logloss:0.18093
[13]	validation_0-logloss:0.13592	validation_1-logloss:0.17993
[14]	validation_0-logloss:0.13412	validation_1-logloss:0.17911
[15]	validation_0-logloss:0.13310	validation_1-logloss:0.17850
[1

In [89]:
import performance

num_round=25
preds = clf.predict(testX)
performance.printStats(testY, preds)

=== Performance Stats ===

Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      8789
           1       0.51      0.04      0.08       558

    accuracy                           0.94      9347
   macro avg       0.73      0.52      0.53      9347
weighted avg       0.92      0.94      0.92      9347


Sensitivity (ability to correctly predict true): 0.044802867383512544
Specificity (ability to correctly predict false): 0.9972693139151212
Informedness (probability of informed decision): 0.04207218129863377
Accuracy: 0.940408687279341
ROC AUC: 0.5210360906493169
